In [1]:
# moduos
import polars as pl

In [ ]:
from typing import List
# Funciones

def get_encoding(filename_in: str, filename_out:str) -> None:
    import chardet

    with open(filename_in, 'rb') as f:
        raw_data = f.read()
        result = chardet.detect(raw_data)
        encoding = result['encoding']

        try:
            decoded_data = raw_data.decode(encoding=encoding)
        except UnicodeDecodeError:
            # Si falla, intenta con una lista de codificaciones comunes
            common_encodings = ['utf-8', 'iso-8859-1', 'cp1252']
            for enc in common_encodings:
                try:
                    decoded_data = raw_data.decode(encoding=enc)
                    break
                except UnicodeDecodeError:
                    continue
            else:
                raise UnicodeDecodeError(f"No se pudo decodificar el archivo con las codificaciones probadas. Último intento fallido con {encoding}")

    with open(filename_out, 'w', encoding='utf-8') as f2:
        f2.write(decoded_data)

def tweak_df(df: pl.DataFrame, columns_trans: list, schema: dict) -> pl.DataFrame:
  # aplicación de funciones para el df
  # acepta lista de expresiones
  return df.with_columns(columns_trans).cast(schema)

def transformations(file_name: str,tweak_list: list,schema_dict: dict) -> pl.DataFrame:
  # decodificando
  get_encoding(f"{file_name}.csv",f"{file_name}_decoded.csv")
  # leyendo el archivo
  df = pl.read_csv(f"{file_name}_decoded.csv")
  # aplicando los cambios al df
  return tweak_df(df,tweak_list,schema_dict)

Nos metimos al IFT (instituto federal de telecomunicaciones) en especial a la sección de servicios móvil de acceso a internet, para delimitar

Subsecciones:
  - Lineas (Ya)
  - Lineas (Serie hitórica desde 2010) (Ya)
  - Tráfico de datos (TB) (Ya)
  - Índice de concentración (IHH) (Ya)
  - Participación de mercado (Serie desde 2010) (Ya)
  - Líneas por cada 100  habitantes (Estatal) (No)
  -Líneas por cada 100 habitantes (Serie historica desde 2010) (No)

## LINEAS
Tabla de información de las líneas de servicio móvil de acceso a internet por esquema de pago (prepago y pospago), serie mensual desde el 2013.

[diccionario de datos](https://docs.google.com/spreadsheets/d/176qSChhhpF43hzslsFscsAcBblJ0Wa_cyBHnzWTq1Eg/edit?gid=0#gid=0)

### To do:
- eda
- encontrar los valores distintos

### Done:
- Ir de acuerdo al esquema en el diccionario de datos



In [ ]:
#Primero vamos a ver lineas
#nos falta un catálogo de datos
!wget https://bit.ift.org.mx/descargas/datos/tabs/TD_LINEAS_INTMOVIL_ITE_VA.csv

In [ ]:
# creación de esquema de acuera al diccionario de datos
schema_lineas = {
    "FECHA": pl.Date,
    "ANIO": pl.Int16,
    "MES": pl.Int8,
    # "K_GRUPO": pl.Categorical,
    "GRUPO": pl.Categorical,
    "K_EMPRESA": pl.Categorical,
    "EMPRESA": pl.Categorical,
    "CONCESIONARIO": pl.Categorical,
    "L_PREPAGO_E": pl.Int32,
    "L_POSPAGO_E": pl.Int32,
    "L_POSPAGOC_E": pl.Int32,
    "L_POSPAGOL_E": pl.Int32,
    "L_NO_ESPECIFICADO_E": pl.Int32,
    "L_TOTAL_E": pl.Int32,
    "FOLIO": pl.Categorical
  }

tweak_columns = [
    pl.col( "L_PREPAGO_E" ).str.replace_all(",","").str.to_integer(base=10),
    pl.col( "L_POSPAGO_E" ).str.replace_all(",","").str.to_integer(base=10),
    pl.col( "L_POSPAGOC_E" ).str.replace_all(",","").str.to_integer(base=10),
    pl.col( "L_POSPAGOL_E" ).str.replace_all(",","").str.to_integer(base=10),
    pl.col( "L_NO_ESPECIFICADO_E" ).str.replace_all(",","").str.to_integer(base=10),
    pl.col( "L_TOTAL_E" ).str.replace_all(",","").str.to_integer(base=10),
    pl.col("FECHA").str.to_date("%d%b%Y"),
    pl.col("FOLIO").cast(pl.Utf8)
  ]

In [ ]:
df_lineas = transformations("TD_LINEAS_INTMOVIL_ITE_VA",tweak_columns,schema_lineas)

In [4]:
from airflow import DAG

In [6]:
help(DAG)

Help on class DAG in module airflow.models.dag:

class DAG(airflow.utils.log.logging_mixin.LoggingMixin)
 |  DAG(dag_id: 'str', description: 'str | None' = None, schedule: 'ScheduleArg' = <airflow.utils.types.ArgNotSet object at 0x7fd645771640>, schedule_interval: 'ScheduleIntervalArg' = <airflow.utils.types.ArgNotSet object at 0x7fd645771640>, timetable: 'Timetable | None' = None, start_date: 'datetime | None' = None, end_date: 'datetime | None' = None, full_filepath: 'str | None' = None, template_searchpath: 'str | Iterable[str] | None' = None, template_undefined: 'type[jinja2.StrictUndefined]' = <class 'jinja2.runtime.StrictUndefined'>, user_defined_macros: 'dict | None' = None, user_defined_filters: 'dict | None' = None, default_args: 'dict | None' = None, concurrency: 'int | None' = None, max_active_tasks: 'int' = 16, max_active_runs: 'int' = 16, max_consecutive_failed_dag_runs: 'int' = 0, dagrun_timeout: 'timedelta | None' = None, sla_miss_callback: 'None | SLAMissCallback | list

In [1]:
from airflow.decorators import dag, task
from airflow import tasm

In [3]:
help(task)

Help on TaskDecoratorCollection in module airflow.decorators object:

class TaskDecoratorCollection(builtins.object)
 |  Implementation to provide the ``@task`` syntax.
 |
 |  Methods defined here:
 |
 |  __getattr__(self, name: 'str') -> 'TaskDecorator'
 |      Dynamically get provider-registered task decorators, e.g. ``@task.docker``.
 |
 |  ----------------------------------------------------------------------
 |  Static methods defined here:
 |
 |  __call__ = python_task(python_callable: 'Callable | None' = None, multiple_outputs: 'bool | None' = None, **kwargs) -> 'TaskDecorator' from airflow.decorators.python
 |      Wrap a function into an Airflow operator.
 |
 |      Accepts kwargs for operator kwarg. Can be reused in a single DAG.
 |
 |      :param python_callable: Function to decorate
 |      :param multiple_outputs: If set to True, the decorated function's return value will be unrolled to
 |          multiple XCom values. Dict will unroll to XCom values with its keys as XCom

In [ ]:
from airflow import DAG
from airflow.decorators import task

with DAG(dag_id="prueba",schedule=None) as dag:
    @task(task_id="primera task", task_display_name="primera task")
    def task_1():
        return "esta es la primera prueba"
